<div style="direction:rtl;line-height:300%;background-color:#282C34;border-radius: 25px;">
	<img src='https://ut.ac.ir/assets/d7a19645/images/logos/loading-logo.png' alt="UT logo" width="130" height="130" align="left" style='padding-top: 15px; padding-left: 20px;'>
	<img src="https://engstudent.ut.ac.ir/wp-content/uploads/Fanni-Transparent.png" width="167" height="125" align="right" style='padding-top: 15px; padding-right: 20px;'>
	<font face="XB Zar" size=5>
		<div align=center>
			<font face="IranNastaliq" size=30 color=white>
				<p></p>
				<p></p>
به نام خدا
			</font>
			<br />
			<br />
			<font color=#1597E5>
دانشگاه تهران - دانشکده مهندسی برق و کامپیوتر
            </font>
            <br />
			<font color=#69DADB>
هوش مصنوعی
            </font>
			<br />
			<font color=#FFA400>
پاییز ۱۴۰۰
		</div>
		<hr color=#6ECB63/>
		<font color=#FF5C58 size=6>
			<div align=center>
پروژه یک - الگوریتم‌های جست‌وجو
            </div>
		</font>
        <br />
		<div align=center>
		<font color=yellow>
حسام اسداله‌زاده 
            <br />
            شماره دانشجویی: ۸۱۰۱۹۸۳۴۶
			</font>
        </div>
		<!hr color=#6ECB63/>
		<style type="text/css" scoped>
        p{
        border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
        };
        </style>            
		
</div>

# Informed and Uninformed Search
Main purpose of this project is to use informed and uninformed searches to find the solution for our problem. The problem we're facing here is finding the optimal way of getting doctors to their last location and collecting all of the mixes in our map.

In [1]:
import collections
import time
import numpy as np

# Defining Constants
In this part constant values are defined in order to increase the readability of the code and making changes more flexible.


In [2]:
DOCTOR = 'DR'
MIX = 'MM'
DOUBLING = 'X2'
FORBIDDEN = 'XX'
EMPTY = '--'
END = 'SS'
FILENAME = 'test1.in'

# Reading Map:
In this part we define a function to read our map from file and create a 2D list which represents our map

In [3]:
def get_input(filename):
    file = open(filename)
    n, m = map(int, file.readline().split())
    input_map = [[EMPTY]*m for _ in range(n)]
    input_map[0][0] = DOCTOR
    input_map[n-1][m-1] = END
    c, k = map(int, file.readline().split())
    for _ in range(c):
        x, y = map(int, file.readline().split())
        if input_map[x][y] == EMPTY:
            input_map[x][y] = MIX
        else:
            input_map[x][y] += (','+MIX)
    for _ in range(k):
        x, y = map(int, file.readline().split())
        if input_map[x][y] == EMPTY:
            input_map[x][y] = DOUBLING
        else:
            input_map[x][y] += (','+DOUBLING)
    d = int(file.readline())
    for _ in range(d):
        x, y = map(int, file.readline().split())
        input_map[x][y] = FORBIDDEN
    file.close()
    return n, m, input_map

## Print map
Also, we define a print function in order to make sure that we've read the map correctly and take a first look at it.

In [4]:
def print_map(n, m, input_map):
    print('|----------'*m, end='|\n')
    for i in range(n):
        print('|    ', end='')
        for j in range(m):
            if len(input_map[n-1-i][j]) == 2:
                print(input_map[n-1-i][j], end='    |    ')
            else:
                print(input_map[n-1-i][j], end=' |    ')
        print('\n', '|----------|', sep='', end='')
        print('----------|'*(m-1), sep='')

# def print_map(n, m, input_map):
#     for i in range(n):
#         for j in range(m):
#             print(input_map[i][j], end='\t')
#         print()

In [5]:
n, m, input_map = get_input(FILENAME)
print_map(n, m, input_map)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|


In [6]:
# forbidden locations are global and don't vary in each state
forbiddens = []

# Modeling the problem

In this problem we considered each **state** consisting of a list of the positions of `doctors`, `mixes`, `doubling` drugs, and the depth (or `cost`) of our state, and of course its `parent` state. The initial state is read from the file. The goal state is obtained, when the *list of mixes is empty and all of doctors are at last location*. Each action can be moving a doctor one unit to the left, right, top, and bottom. If after an action doctor meets a `mix`, he must eat it on that action. If after an action doctor meets a `doubling`, he must eat it on that action and a doctor should be added to `(n-1, 0)`.

In [7]:
class State:
    def __init__(self, doctors, mixes, doublings, parent = None, pathcost = 0):
        self.doctors = doctors
        self.mixes = mixes
        self.doublings = doublings
        self.path_cost = pathcost
        self.parent = parent

    def add_doctor(self):
        self.doctors.append((n-1, 0))

    def get_child(self):
        new_doctors = self.doctors[:]
        new_mixes = self.mixes[:]
        new_doublings = self.doublings[:]
        return State(new_doctors, new_mixes, new_doublings, self, self.path_cost+1)

    def __str__(self) -> str:
        ret = ''
        ret += '\t\tdoctors: ' + str(self.doctors) + '\n'
        ret += '\t\tmixes: ' + str(self.mixes) + '\n'
        ret += '\t\tdoublings: ' + str(self.doublings) + '\n'
        ret += '\t\tpath cost: ' + str(self.path_cost) + '\n'
        return ret
        
    def get_string(self, A_star = False):
        ret = ''
        ret += str(self.doctors)[2:-2]
        ret += str(self.mixes)[2:-2]
        ret += str(self.doublings)[2:-2]
        if(A_star):
        #    ret += str(self.path_cost)
            ret += str(self.path_cost)
        return ret
    
    def __lt__(self, other):
        return self.path_cost + get_heuristic(self) < other.path_cost + get_heuristic(other)

`get_string()` use **positions** to distinguish between different states.

## Creating initial state

In [8]:
def create_initial_state(n, m, input_map):
    doctors = []
    mixes = []
    doublings = []
    for i in range(n):
        for j in range(m):
            if(DOCTOR in input_map[i][j]):
                doctors.append((i, j))
            if(MIX in input_map[i][j]):
                mixes.append((i, j))
            if(DOUBLING in input_map[i][j]):
                doublings.append((i, j))
            if(FORBIDDEN in input_map[i][j]):
                forbiddens.append((i, j))
    state = State(doctors, mixes, doublings)
    if(MIX in input_map[0][0]):
        state.mixes.remove((0, 0))
    if(DOUBLING in input_map[0][0]):
        state.doublings.remove((0, 0))
        state.add_doctor()
    return state

# Actions

For each doctor there are 4 available directions which means there are at most ${({4}\times{\# ofDoctors})}$ children for each state.

The implementation of action functions are shown below. on each action function call, a new location for doctor is calculated. If the new location is verified by `validate_new_location` function, `apply_action` function will be called to get a new state based on the new position. At the end, new state is returned if it's validated otherwise None is returned.

In [9]:
def validate_new_location(n, m, new_x, new_y):
    if new_x < 0 or new_x >= n:
        return False
    if new_y < 0 or new_y >= m:
        return False
    if (input_map[new_x][new_y] == FORBIDDEN):
        return False
    return True

In [10]:
def apply_action(state, old_x, old_y, new_x, new_y):
    if((new_x, new_y) in state.mixes):
        state.mixes.remove((new_x, new_y))

    elif((new_x, new_y) in state.doublings):
        state.doublings.remove((new_x, new_y))
        state.add_doctor()
    
    state.doctors.remove((old_x, old_y))
    state.doctors.append((new_x, new_y))
    return state

In [11]:
def action_right(state, old_x, old_y):
    new_x = old_x
    new_y = old_y + 1
    
    if validate_new_location(n, m, new_x, new_y):
        new_state = state.get_child()
        new_state = apply_action(new_state, old_x, old_y, new_x, new_y)
        return new_state
    else:
        return None

In [12]:
def action_left(state, old_x, old_y):
    new_x = old_x
    new_y = old_y - 1
    
    if validate_new_location(n, m, new_x, new_y):
        new_state = state.get_child()
        new_state = apply_action(new_state, old_x, old_y, new_x, new_y)
        return new_state
    else:
        return None

In [13]:
def action_up(state, old_x, old_y):
    new_x = old_x - 1
    new_y = old_y
    
    if validate_new_location(n, m, new_x, new_y):
        new_state = state.get_child()
        new_state = apply_action(new_state, old_x, old_y, new_x, new_y)
        return new_state
    else:
        return None

In [14]:
def action_down(state, old_x, old_y):
    new_x = old_x + 1
    new_y = old_y
    
    if validate_new_location(n, m, new_x, new_y):
        new_state = state.get_child()
        new_state = apply_action(new_state, old_x, old_y, new_x, new_y)
        return new_state
    else:
        return None

In [15]:
def make_actions(state):
    children = []
    actions = [action_right, action_down, action_left, action_up]
    
    for dr in state.doctors:
        for action in actions:
            if (dr[0] == n-1) and (dr[1] == m-1):
                continue
            child_state = action(state, dr[0], dr[1])
            if(child_state is not None):
                children.append(child_state)

    return children

# Goal State
Goal State is a state which includes no mixes and all doctors are at their end position.

In [16]:
def is_goal(state):
    for dr in state.doctors:
        if dr[0] != n-1 or dr[1] != m-1:
            return False
    if len(state.mixes) != 0:
        return False
    return True

# Path Cost
Path cost is defined 1 for all edges

# BFS
Breadth-first search is a simple strategy in which the root node is expanded first, then all the successors of the root node are expanded next, then their successors, and so on. In general, all the nodes are expanded at a given depth in the search tree before any nodes at the next level are expanded.

We use python's built-in `set` and `deque` from `collections` library in order to implement **Breadth-first search** algorithm

In [17]:
def bfs(initial_state):
    states_count = 1
    
    if(is_goal(initial_state)):
        return initial_state, states_count
    
    frontier = collections.deque([initial_state])
    explored = set()
    
    explored.add(initial_state.get_string())
    
    while(frontier):
        state = frontier.popleft()
        children = make_actions(state)
        
        for child_state in children:
            states_count += 1
            if(child_state.get_string() not in explored): 
                if(is_goal(child_state)):
                    return child_state, states_count
                else:
                    explored.add(child_state.get_string())
                    frontier.append(child_state)

# IDS
Also, We've implemented Iterative Deepening (depth-first) Search Algorithm. It uses `dls` function which is **depth-limited** depth-first search. `dls` uses stack as the frontier data structure. By using stack, state's child states are explored in the way that always the deepest state is explored. by using dls we have **a limit on depth**, if new state's depth is more than specified depth, it's not added to frontier. We also consider each state's depth (or cost) to distinguish between duplicate states since an state in dfs can be explored with lower depth if there's a lower path to state in another branch of the starting state's node.

In [18]:
ids_states_count = 0
ids_uniques_count = 0
def dls(depth, initial_state):
    global ids_states_count
    global ids_uniques_count
    
    if is_goal(initial_state):
        return initial_state
    
    ids_states_count += 1
    ids_uniques_count += 1

    frontier = collections.deque([initial_state])
    explored = set()
    
    while(frontier):
        state = frontier.pop()
        explored.add(state.get_string() + str(state.path_cost))
        
        if(is_goal(state)):
            return state
        
        if(state.path_cost < depth):
            children = make_actions(state)

            for child_state in children:
                ids_states_count += 1
                if((child_state.get_string() + str(child_state.path_cost)) not in explored): 
                    ids_uniques_count += 1
                    frontier.append(child_state)

In [19]:
def ids(initial_state):
    global ids_states_count
    global ids_uniques_count
    ids_states_count = 0
    ids_uniques_count = 0

    result = None
    depth = 0
    
    while result is None:
        result = dls(depth, initial_state)
        depth += 1
    
    return (result, ids_states_count, ids_uniques_count)

# A* 
A* is an informed search algorithm which evaluates nodes by combining $g(n)$, the cost to reach the node, and $h(n)$ our heuristic function which estimates the cost of reaching the goal from a state.
$$ f(n) = g(n) + h(n) $$
The heuristic function chosen in this problem is **sum of the (number of remining mixes in the map) and min(each doctor's manhattan distance from their end position or nearest mix).**
$$h(n) = \#ofRemainingMixes + \sum\limits_{dr} min(ManhattanDist(dr, (n-1, m-1)), min(ManhattanDist(dr, mixes))) $$
If any of the mixes is remaining on the map, the $cost$ to reach goal is definitely bigger than reaching that specific mix. If more than a mix are remaining, $cost$ is obviously bigger than distance of agent from nearest mix plus number of remaining mixes. If one single mix is remaining, then the cost to reach goal is bigger or equal to our heuristic. So, $h(n)$ never overestimates the cost to reach the goal.

$$ \forall{n}, h(n) \leq {h}^{*}(n) $$

**So, heuristic is Admissible.**


In [20]:
import math

In [21]:
def manhattan_distance(loc1, loc2):
    return abs(loc1[0] - loc2[0]) + abs(loc1[1] - loc2[1])

In [22]:
def sum_doctors_dist(state):
    sum_ = 0
    for dr in state.doctors:
        sum_ += manhattan_distance(dr, (n-1, m-1))
    return sum_

In [23]:
def get_heuristic(state):
    global n, m
    _sum = 0
    for dr in state.doctors:
        _min = manhattan_distance(dr, (n-1, m-1))
        for mix in state.mixes:
            dist = manhattan_distance(dr, mix)
            if dist < _min:
                _min = dist
        _sum += _min
    return _sum + len(state.mixes)

We use `heappop` from `heapq` and `insort` from `bisect` in order to implement our A* Algorithm

In [24]:
import heapq
import bisect

In [25]:
def A_star(initial_state):
    states_count = 1
    if(is_goal(initial_state)):
        return initial_state, states_count, states_count
        
    frontier = []
    bisect.insort(frontier, initial_state)
    
    explored = set()
    explored.add(initial_state.get_string(True))
    
    while(len(frontier)):
        state = heapq.heappop(frontier)
        children = make_actions(state)
        
        for child_state in children:
            states_count += 1
            if(child_state.get_string(True) not in explored):  
                if(is_goal(child_state)):
                    return child_state, states_count, len(explored)
                    
                else:
                    explored.add(child_state.get_string(True))
                    bisect.insort(frontier, child_state)

# Testing Algorithms

In [26]:
def print_state(state):
    global forbiddens
    _map = [[EMPTY]*m for _ in range(n)]
    for dr in state.doctors:
        if _map[dr[0]][dr[1]] == EMPTY:
            _map[dr[0]][dr[1]] = DOCTOR
        else:
            _map[dr[0]][dr[1]] += (','+DOCTOR)
    for mix in state.mixes:
        if _map[mix[0]][mix[1]] == EMPTY:
            _map[mix[0]][mix[1]] = MIX
        else:
            _map[mix[0]][mix[1]] += (','+MIX)
    for dblng in state.doublings:
        if _map[dblng[0]][dblng[1]] == EMPTY:
            _map[dblng[0]][dblng[1]] = DOUBLING
        else:
            _map[dblng[0]][dblng[1]] += (','+DOUBLING)
    for forbidden in forbiddens:
        _map[forbidden[0]][forbidden[1]] = FORBIDDEN
    print('|----------'*m, end='|\n')
    for i in range(n):
        print('|    ', end='')
        for j in range(m):
            if len(_map[n-1-i][j]) == 2:
                print(_map[n-1-i][j], end='    |    ')
            else:
                print(_map[n-1-i][j], end=' |    ')
        print('\n', '|----------|', sep='', end='')
        print('----------|'*(m-1), sep='')
    print('*******************************************************************************')

In [27]:
def run_test(test_num):
    global n, m, input_map, forbiddens
    FILENAME = 'test'+str(test_num)+'.in'
    n, m, input_map = get_input(FILENAME)
    print_map(n, m, input_map)
    forbiddens = []
    print("TESTING "+FILENAME+':')
    print('BFS Algorithm\n')
    initial_state = create_initial_state(n, m, input_map)
    tic = time.time()
    bfs_last_state, states_count = bfs(initial_state)
    toc = time.time()
    print('\tFinal State: ')
    print(bfs_last_state)
    print('\ttime spent = ', '{:.2f}'.format((toc-tic)*1000), 'ms = ', '{:.2f}'.format(toc-tic), 's', sep='')
    print('\tcount of seen states:', states_count)
    print('\nIDS Algorithm\n')
    initial_state = create_initial_state(n, m, input_map)
    tic = time.time()
    ids_last_state, ids_states_count, ids_uniques_count = ids(initial_state)
    toc = time.time()
    print('\tFinal State: ')
    print(ids_last_state)
    print('\ttime spent = ', '{:.2f}'.format((toc-tic)*1000), 'ms = ', '{:.2f}'.format(toc-tic), 's', sep='')
    print('\tcount of seen states:', ids_states_count, '\n\tcount of unique seen states:', ids_uniques_count,'\n')
    print('\nA* Algorithm\n')
    initial_state = create_initial_state(n, m, input_map)
    tic = time.time()
    Astar_last_state, i, j = A_star(initial_state)
    toc = time.time()
    print('\tFinal State: ')
    print(Astar_last_state)
    print('\ttime spent = ', '{:.2f}'.format((toc-tic)*1000), 'ms = ', '{:.2f}'.format(toc-tic), 's', sep='')
    print('\tcount of seen states:', i, '\n\tcount of explored states:', j,'\n')
    return bfs_last_state, ids_last_state, Astar_last_state

In [28]:
bfs_s, ids_s, astar_s = run_test(1)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
TESTING test1.in:
BFS Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: []
		path cost: 11

	time spent = 29.92ms = 0.03s
	count of seen states: 3946

IDS Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: []
		path cost: 11

	time spent = 230.38ms = 0.23s
	count of seen states: 32738 
	count of unique seen states: 8842 


A* Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: []
		path cost: 11

	time spent = 41.89ms = 0.04s
	count of seen states: 1390 
	count of explored states: 

In [29]:
def print_path(state, heu = False):
    if state.parent:
        print_path(state.parent, heu)
    if heu:
        print('Heuristic =', get_heuristic(state))
    print_state(state)

In [30]:
print_path(bfs_s)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
*******************************************************************************
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    --    |    DR    |    MM    |    --    |    
|----------|----------|----------|----------|
************************************************************

In [31]:
print_path(ids_s)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
*******************************************************************************
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    --    |    DR    |    MM    |    --    |    
|----------|----------|----------|----------|
************************************************************

In [32]:
print_path(astar_s, True)

Heuristic = 4
|----------|----------|----------|----------|
|    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
*******************************************************************************
Heuristic = 4
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    --    |    DR    |    MM    |    --    |    
|----------|----------|----------|----------|
********************************

In [33]:
bfs_s, ids_s, astar_s = run_test(2)

|----------|----------|----------|----------|
|    --    |    X2    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR,X2 |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
TESTING test2.in:
BFS Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: [(1, 1), (3, 1)]
		path cost: 11

	time spent = 6828.73ms = 6.83s
	count of seen states: 958125

IDS Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: [(1, 1), (3, 1)]
		path cost: 11

	time spent = 18977.24ms = 18.98s
	count of seen states: 2459370 
	count of unique seen states: 445722 


A* Algorithm

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: [(1, 1), (3, 1)]
		path cost: 11

	time spent = 223.40ms = 0.22s
	c

In [34]:
print_path(bfs_s)

|----------|----------|----------|----------|
|    DR    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
*******************************************************************************
|----------|----------|----------|----------|
|    DR    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    --    |    DR    |    --    |    XX    |    
|----------|----------|----------|----------|
************************************************************

In [35]:
print_path(ids_s)

|----------|----------|----------|----------|
|    DR    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
*******************************************************************************
|----------|----------|----------|----------|
|    --    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    DR    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
************************************************************

In [36]:
print_path(astar_s, True)

Heuristic = 6
|----------|----------|----------|----------|
|    DR    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
*******************************************************************************
Heuristic = 6
|----------|----------|----------|----------|
|    --    |    X2    |    --    |    --    |    
|----------|----------|----------|----------|
|    DR    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR    |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
********************************

In [37]:
bfs_s, ids_s, astar_s = run_test(3)

|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
TESTING test3.in:
BFS Algorithm

	Final State: 
		doctors: [(5, 5), (5, 5)]
		mixes: []
		do

In [38]:
print_path(bfs_s)

|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
*******************************************************************************
|----------|

In [39]:
print_path(ids_s)

|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
*******************************************************************************
|----------|

In [40]:
print_path(astar_s, True)

Heuristic = 8
|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
******************************************************************************

# Weighted A*

In this version of A*, heuristic is no longer admissible because first solution found is not guaranteed to be optimal. This algorithm trades off optimality for speed.

In [41]:
ALPHA = 1.5
def get_heuristic(state):
    global ALPHA
    return ALPHA * (len(state.mixes) + sum_doctors_dist(state))

In [42]:
def run_test(test_num):
    global n, m, input_map, forbiddens, ALPHA
    FILENAME = 'test'+str(test_num)+'.in'
    n, m, input_map = get_input(FILENAME)
    print_map(n, m, input_map)
    forbiddens = []
    print("TESTING "+FILENAME+':')
    print('\nWeighted A* Algorithm\tAlpha =', ALPHA,'\n')
    initial_state = create_initial_state(n, m, input_map)
    tic = time.time()
    last_state, i, j = A_star(initial_state)
    toc = time.time()
    print('\tFinal State: ')
    print(last_state)
    print('\ttime spent = ', '{:.2f}'.format((toc-tic)*1000), 'ms = ', '{:.2f}'.format(toc-tic), 's', sep='')
    print('\tcount of seen states:', i, '\n\tcount of explored states:', j,'\n')

## Low Alpha (1.5)

In [43]:
run_test(1)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
TESTING test1.in:

Weighted A* Algorithm	Alpha = 1.5 

	Final State: 
		doctors: [(3, 3)]
		mixes: []
		doublings: [(0, 1)]
		path cost: 12

	time spent = 1.99ms = 0.00s
	count of seen states: 150 
	count of explored states: 90 



In [44]:
run_test(2)

|----------|----------|----------|----------|
|    --    |    X2    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR,X2 |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
TESTING test2.in:

Weighted A* Algorithm	Alpha = 1.5 

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: [(1, 1), (3, 1)]
		path cost: 11

	time spent = 1.00ms = 0.00s
	count of seen states: 48 
	count of explored states: 45 



In [45]:
run_test(3)

|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
TESTING test3.in:

Weighted A* Algorithm	Alpha = 1.5 

	Final State: 
		doctors: [(5, 5)]
		

In [46]:
ALPHA = 5

## High Alpha (5)

In [47]:
run_test(1)

|----------|----------|----------|----------|
|    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    --    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    --    |    XX    |    --    |    
|----------|----------|----------|----------|
|    DR    |    X2    |    MM    |    --    |    
|----------|----------|----------|----------|
TESTING test1.in:

Weighted A* Algorithm	Alpha = 5 

	Final State: 
		doctors: [(3, 3)]
		mixes: []
		doublings: [(0, 1)]
		path cost: 12

	time spent = 1.00ms = 0.00s
	count of seen states: 104 
	count of explored states: 68 



In [48]:
run_test(2)

|----------|----------|----------|----------|
|    --    |    X2    |    --    |    SS    |    
|----------|----------|----------|----------|
|    MM    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|
|    --    |    X2    |    MM    |    XX    |    
|----------|----------|----------|----------|
|    DR,X2 |    --    |    --    |    XX    |    
|----------|----------|----------|----------|
TESTING test2.in:

Weighted A* Algorithm	Alpha = 5 

	Final State: 
		doctors: [(3, 3), (3, 3)]
		mixes: []
		doublings: [(1, 1), (3, 1)]
		path cost: 11

	time spent = 1.02ms = 0.00s
	count of seen states: 43 
	count of explored states: 42 



In [49]:
run_test(3)

|----------|----------|----------|----------|----------|----------|
|    --    |    MM    |    --    |    --    |    --    |    SS    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    X2    |    MM    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    XX    |    --    |    --    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    --    |    X2    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
|    --    |    XX    |    MM    |    --    |    --    |    MM    |    
|----------|----------|----------|----------|----------|----------|
|    DR    |    --    |    XX    |    --    |    --    |    --    |    
|----------|----------|----------|----------|----------|----------|
TESTING test3.in:

Weighted A* Algorithm	Alpha = 5 

	Final State: 
		doctors: [(5, 5)]
		mi

results for test1, test2, test3 respectively:

<img src='table.jpg'>

# Conclusion
As we can see, BFS and IDS algorithms always find the optimal solution if path costs are equal to 1. Both BFS and IDS algorithms have time complexity of $O( b^d )$. IDS takes more time in comparison to BFS since It revisits some states on each iteration of algorithm on specified depth. On the other hand IDS only uses memory space of $O(b.d)$ which is too much more efficient than BFS which uses $O( b^d )$ memory space. A* with an *admissible and consistent heuristic* always finds the optimal path in minimum steps. In all tests A* has expanded less states to find the optimal path. Due to optimality in all algorithms number of steps to reach the goal is equal to the optimal number of steps to reach the goal which is shown in tables.